In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull

# Configuration parameters
GRID_SIZE = 100  # Size of the coordinate plane
MAX_STEP = 1.0   # Maximum step size per axis

# Target points should be provided by the user
TARGETS = np.array([[30.0, 30.0], [70.0, 30.0], [50.0, 70.0]], dtype=float)  # Example targets

# Precompute convex hull and target pairs
hull = ConvexHull(TARGETS)
hull_vertices = TARGETS[hull.vertices]
pairs = [(i, j) for i in range(len(TARGETS)) for j in range(i + 1, len(TARGETS))]

def is_inside_hull(point, hull_vertices):
    """Check if a point is inside the convex hull"""
    n = len(hull_vertices)
    for i in range(n):
        A = hull_vertices[i]
        B = hull_vertices[(i + 1) % n]
        vec_AB = B - A
        vec_AP = point - A
        if np.cross(vec_AB, vec_AP) <= 0:
            return False
    return True

def calculate_step_vector(point, targets, hull_vertices, pairs):
    """Calculate movement vector with random step size based on region"""
    point = np.array(point, dtype=float)
    
    # Check if point is inside the convex hull (bounded region)
    if is_inside_hull(point, hull_vertices):
        # Find the closest line among all pairs
        min_dist = float('inf')
        closest_line = None
        for i, j in pairs:
            A = targets[i]
            B = targets[j]
            vec_AB = B - A
            cross = np.cross(vec_AB, point - A)
            dist = abs(cross) / np.linalg.norm(vec_AB)
            if dist < min_dist:
                min_dist = dist
                closest_line = (A, B)
        
        # Compute projection onto the closest line
        A, B = closest_line
        vec_AB = B - A
        t = np.dot(point - A, vec_AB) / np.dot(vec_AB, vec_AB)
        Q = A + t * vec_AB
        direction = Q - point
    else:
        # Unbounded region: move towards closest target
        distances = np.linalg.norm(targets - point, axis=1)
        closest_idx = np.argmin(distances)
        direction = targets[closest_idx] - point
    
    # Normalize direction if not zero
    norm = np.linalg.norm(direction)
    if norm > 1e-8:
        direction /= norm
    else:
        direction = np.zeros(2)
    
    # Generate random step size (max Euclidean length sqrt(2) handled by per-axis clip)
    random_scale = np.random.uniform(0, MAX_STEP)
    step = direction * random_scale
    
    # Clip step per axis to ensure max step length constraint
    return np.clip(step, -MAX_STEP, MAX_STEP)

# Initialize point cloud
points = np.random.rand(50000, 2).astype(float) * GRID_SIZE

# Perform iterative movements
for _ in range(20):
    for i in range(len(points)):
        step = calculate_step_vector(points[i], TARGETS, hull_vertices, pairs)
        points[i] = np.clip(points[i] + step, 0.0, float(GRID_SIZE))

# Visualization
plt.figure(figsize=(12, 6))

# Point cloud visualization
plt.subplot(121)
plt.scatter(points[:, 0], points[:, 1], s=1, alpha=0.5, c='blue')
plt.scatter(TARGETS[:, 0], TARGETS[:, 1], c='red', s=50, marker='X')
plt.title('Point Cloud Distribution')
plt.xlim(0, GRID_SIZE)
plt.ylim(0, GRID_SIZE)

# Vector field visualization
plt.subplot(122)
xx, yy = np.meshgrid(np.linspace(0, GRID_SIZE, 15), np.linspace(0, GRID_SIZE, 15))
vectors = np.array([calculate_step_vector([x, y], TARGETS, hull_vertices, pairs) 
                    for x, y in zip(xx.ravel(), yy.ravel())])
plt.quiver(xx, yy, vectors[:, 0], vectors[:, 1], scale=25, color='green', alpha=0.7)
plt.scatter(TARGETS[:, 0], TARGETS[:, 1], c='red', s=50, marker='X')
plt.title('Movement Vector Field')
plt.xlim(0, GRID_SIZE)
plt.ylim(0, GRID_SIZE)

plt.tight_layout()
plt.savefig('point_cloud_result.png')

KeyboardInterrupt: 